In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.io.vasp.outputs import Vasprun
import re
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import correlate, find_peaks
from scipy.ndimage import gaussian_filter
%matplotlib ipympl

## Helper functions

In [ ]:
def get_traj(paths, discard=100):
    """
    Get MD trajectory from a single of mutiple vasprun files.
    
    Args:
        paths: path to vasprun.xml, or list of paths to multiple vasprun.xml's.
        discard: number of initial steps to discard.
    """
    if type(paths)==str:
        paths  = [paths]

    traj = []

    for i, path in enumerate(paths):
        if i==0:
            run = Vasprun(path, 
                        ionic_step_offset=discard,
                        parse_dos=False,
                        parse_eigen=False)
        else:
            run = Vasprun(path, 
                    ionic_step_offset=0,
                    parse_dos=False,
                    parse_eigen=False)
        traj += [j['structure'].cart_coords for j in run.ionic_steps]
    
    return(np.asarray(traj))

In [ ]:
def get_energies(paths):
    """
    Get the kinetic and potential energies along an MD trajectory from a single of mutiple vasprun files.
    
    Args:
        paths: path to vasprun.xml, or list of paths to multiple vasprun.xml's.
    """
    if type(paths)==str:
        paths  = [paths]

    kin = []
    pot = []
    tot = []

    for i, path in enumerate(paths):
        run = Vasprun(path, 
                ionic_step_offset=0,
                parse_dos=False,
                parse_eigen=False)
        kin += [j['kinetic'] for j in run.ionic_steps]
        pot += [j['e_wo_entrp'] for j in run.ionic_steps]
        tot += [j['total'] for j in run.ionic_steps]
    
    return((np.asarray(kin), np.asarray(pot), np.asarray(tot)))

In [ ]:
def get_time_step(path):
    """
    Get time step in fs:
    """
    if type(path)==list:
        path = path[0]
    time_step = None
    with open(path, 'r') as file:
        for line in file.readlines():
            if 'POTIM' in line:
                time_step = float(re.findall('\d+.\d+', line)[0])
    
    return(time_step)

In [ ]:
def get_velocities(traj, time_step, scheme='forward'):
    """
    Calculate velocities from an MD trajectory using finite-differences.
    """
    n_steps = len(traj)
    vel = np.zeros((n_steps-1, len(traj[0]), len(traj[0,0])))
    for i in range(n_steps-1):
        vel[i] = (traj[i+1] - traj[i]) / time_step
        
    return(vel)

In [ ]:
def get_dos(path):
    """
    Get Phonopy DOS from a dat file generated by Phonopy.
    """
    dos = []
    with open(path, 'r') as file:
        for line in file.readlines()[1:]:
            dos.append([float(i) for i in line.split()])
    
    return np.array(dos)

## Energies

In [ ]:
path = "/work/08697/sohmapte/ls6/MD/02-MD/7/vasprun.xml"
# path = ["/work/08697/sohmapte/ls6/MD/02-MD/5/vasprun_1.xml", "/work/08697/sohmapte/ls6/MD/02-MD/5/vasprun_2.xml"]

kin, pot, tot = get_energies(path)
time_step = get_time_step(path)
n_steps = len(tot)
time = np.linspace(0,time_step*n_steps,n_steps)

kB = 8.617332478E-5     # Boltzmann Constant in [eV/K]
T = 2*kin/(kB * 3*(64-1))

In [ ]:
plt.close()
plt.plot(time, T)
window_size = 100
plt.plot(time[window_size-1:], np.convolve(T, np.ones(window_size)/window_size, mode='valid'))
plt.xlabel("Time (fs)")
# plt.ylabel("Total energy (eV)")
plt.ylabel("Temperature (K)")

## Auto-correlation function

In [ ]:
# Get trajectory and calculate velocities.

path = "/work/08697/sohmapte/ls6/MD/02-MD/6/vasprun.xml"
# path = ["/work/08697/sohmapte/ls6/MD/02-MD/5/vasprun_1.xml", "/work/08697/sohmapte/ls6/MD/02-MD/5/vasprun_2.xml"]

traj = get_traj(path, discard=6000)
time_step = get_time_step(path)
n_steps = len(traj)
time = np.linspace(0,time_step*n_steps,n_steps)

vel = get_velocities(traj, time_step)

In [ ]:
# Calculating velocity auto-correlation function.
vaf = np.zeros(2*n_steps-3)
for i in range(len(vel[0])):
    for j in range(len(vel[0,0])):
        c = correlate(vel[:,i,j], vel[:,i,j])
        vaf += c
vaf /= np.sum(vel**2)

# Fourier transforming VAF to get phonon DOS.
vaf_fft = fft(vaf)
freq = fftfreq(2*n_steps-3,time_step) * 1E3         # In THz
# freq *= 33.365                                      # Convert to cm-1
dos = np.abs(vaf_fft)
sigma = 3                                           # Standard deviation for the Gaussian filter.
dos = gaussian_filter(dos, sigma)                   # Applying a Gaussian filter to broaden the DOS.

# Phonon DOS using finite difference method from Phonopy.
dos_phonopy = get_dos("/work/08697/sohmapte/ls6/MD/03-Phonons/3x3x3/total_dos.dat")
dos_phonopy[:,1] *= np.max(dos) / np.max(dos_phonopy[:,1])  # Normalizing dos_phonopy with dos

In [ ]:
# Calculating velocity auto-correlation function using direct integration.
vaf2 = np.zeros(n_steps-1)
for tau in range(n_steps-1):
    for n in range(n_steps-tau-1):
        vaf2[tau] += np.sum(vel[n,:,:] * vel[n+tau,:,:])
vaf2 /= n_steps

vaf2_fft = fft(vaf2)
dos2 = np.abs(vaf2_fft)
freq2 = fftfreq(n_steps-1,time_step) * 1E3

In [ ]:
plt.close()

# Plotting real time VAF:
# plt.plot(time[1:-1], vaf[c.size//2+1:])        # Only plotting t>0.
# plt.plot(time[1:-1], vaf2[1:], alpha=0.8)
# plt.xlabel("Time (fs)")

# Plotting frequency space VAF:
plt.plot(freq[:n_steps-1], dos[:n_steps-1], label='MD')
# plt.plot(freq2[:n_steps//2], dos2[:n_steps//2], label='MD from direct integration')
plt.plot(dos_phonopy[:,0], dos_phonopy[:,1], label='finite difference using Phonopy')
plt.xlabel('Frequency (THz)')
plt.ylabel("Phonon DOS")
plt.xlim(0,18)
plt.legend()

plt.axhline(y=0.0, color='k', linewidth=1, linestyle='--')

In [ ]:
# Calculating peak frequencies from DOS.
peaks = find_peaks(dos[:n_steps-1], 2)[0]
peak_freqs = freq[peaks]